# Final Project
### Max Fleming and Kunal Renjen
Fantasy Football Data from 2017-2022
Questions to answer:
* Who are the Rushing, Passing and Recieving Yards Leader statistic by Year?
* Using data from 2017-2021, are there specific metrics to predict what players to use in the next year (2022)?



Importing necessary python libraries and mounting google drive

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from scipy.stats import pearsonr
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Reading the dataset in and printing the first five rows of data

In [2]:
fantasy_data = pd.read_csv('/content/drive/MyDrive/Intro Data Science Fall 2024/Final Project/fantasy_merged_7_17.csv')
fantasy_data.head()

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,Rec,RecYds,YR,RecTD,Fmb,FL,PPR,PlayerID,PosRk,Year
0,1,Todd Gurley,LAR,RB,23,15,15,0,0,0,...,64,788,12.31,6,5,2,383.3,GurlTo01,1.0,2017
1,2,Russell Wilson,SEA,QB,29,16,16,339,553,3983,...,0,0,NaN,0,14,3,347.9,WilsRu00,1.0,2017
2,3,Le'Veon Bell,PIT,RB,25,15,15,0,0,0,...,85,655,7.71,2,3,2,341.6,BellLe00,2.0,2017
3,4,Alvin Kamara,NOR,RB,22,16,3,0,0,0,...,81,826,10.20,5,1,1,320.4,KamaAl00,3.0,2017
4,5,Antonio Brown,PIT,WR,29,14,14,0,0,0,...,101,1533,15.18,9,4,0,310.3,BrowAn04,1.0,2017


Examining the titles of all the columns in the dataset for further reference

In [3]:
fantasy_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3388 entries, 0 to 3387
Data columns (total 27 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Rk        3388 non-null   int64  
 1   Player    3388 non-null   object 
 2   Tm        3388 non-null   object 
 3   FantPos   3388 non-null   object 
 4   Age       3388 non-null   int64  
 5   G         3388 non-null   int64  
 6   GS        3388 non-null   int64  
 7   Cmp       3388 non-null   int64  
 8   Att       3388 non-null   int64  
 9   Yds       3388 non-null   int64  
 10  TD        3388 non-null   int64  
 11  Int       3388 non-null   int64  
 12  RushAtt   3388 non-null   int64  
 13  RushYds   3388 non-null   int64  
 14  YA        2017 non-null   float64
 15  RushTD    3388 non-null   int64  
 16  Tgt       3388 non-null   int64  
 17  Rec       3388 non-null   int64  
 18  RecYds    3388 non-null   int64  
 19  YR        2879 non-null   float64
 20  RecTD     3388 non-null   int6

Renaming the Yds column to PassingYds. This will help with analysis later when seperating data for Quarterbacks, Running Backs and Wide Recievers

In [4]:
fantasy_data.rename(columns={'Yds': 'PassingYds'}, inplace=True)

In [5]:
fantasy_data['PassingYds/Int'] = fantasy_data['PassingYds'] / fantasy_data['Int']
fantasy_data['RushYds/Fmb'] = fantasy_data['RushYds'] / fantasy_data['Fmb']
fantasy_data['CompPct'] = fantasy_data['Cmp'] / fantasy_data['Att']
fantasy_data['RushYds/TD'] = fantasy_data['RushYds'] / fantasy_data['RushTD']
fantasy_data['RecYds/TD'] = fantasy_data['RecYds'] / fantasy_data['RecTD']
fantasy_data.head()

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,PassingYds,...,FL,PPR,PlayerID,PosRk,Year,PassingYds/Int,RushYds/Fmb,CompPct,RushYds/TD,RecYds/TD
0,1,Todd Gurley,LAR,RB,23,15,15,0,0,0,...,2,383.3,GurlTo01,1.0,2017,NaN,261.000000,NaN,100.384615,131.333333
1,2,Russell Wilson,SEA,QB,29,16,16,339,553,3983,...,3,347.9,WilsRu00,1.0,2017,362.090909,41.857143,0.61302,195.333333,NaN
2,3,Le'Veon Bell,PIT,RB,25,15,15,0,0,0,...,2,341.6,BellLe00,2.0,2017,NaN,430.333333,NaN,143.444444,327.500000
3,4,Alvin Kamara,NOR,RB,22,16,3,0,0,0,...,1,320.4,KamaAl00,3.0,2017,NaN,728.000000,NaN,91.000000,165.200000
4,5,Antonio Brown,PIT,WR,29,14,14,0,0,0,...,0,310.3,BrowAn04,1.0,2017,NaN,0.000000,NaN,NaN,170.333333


Creation of new dataframes that only contain Running Backs, Quarterbacks and Wide Recievers with data from 2017-2022.

In [6]:
rb_data = fantasy_data[fantasy_data['FantPos'] == 'RB']
qb_data = fantasy_data[fantasy_data['FantPos'] == 'QB']
wr_data = fantasy_data[fantasy_data['FantPos'] == 'WR']

Distribution of Running Backs by Rushing Yards

In [9]:
# Create the histogram
fig = go.Figure(data=[go.Histogram(x=rb_data['RushYds'], nbinsx=50, marker_color='red')])

# Update the layout
fig.update_layout(
    title='Distribution of Rushing Yards',
    xaxis_title='Rushing Yards',
    yaxis_title='Count'
)

# Show the plot
fig.show()

 #Code to Convert to HTML for github
fig.write_html("Distribution_of_Rushing_Yards.html")
from google.colab import files
files.download('Distribution_of_Rushing_Yards.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Distribution of Wide Recievers by Receiving Yards

In [10]:
# Create the histogram
fig = go.Figure(data=[go.Histogram(x=wr_data['RecYds'], nbinsx=50, marker_color='green')])

# Update the layout
fig.update_layout(
    title='Distribution of Receiving Yards',
    xaxis_title='Receiving Yards',
    yaxis_title='Count'
)

# Show the plot
fig.show()

fig.write_html("Distribution_of_Receiving_Yards.html")
from google.colab import files
files.download('Distribution_of_Receiving_Yards.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Distribution of Quarterbacks by Passing Yards

In [11]:
# Create the histogram
fig = go.Figure(data=[go.Histogram(x=qb_data['PassingYds'], nbinsx=50, marker_color='blue')])

# Update the layout
fig.update_layout(
    title='Distribution of Passing Yards',
    xaxis_title='Passing Yards',
    yaxis_title='Count'
)

# Show the plot
fig.show()

fig.write_html("Distribution_of_Passing_Yards.html")
from google.colab import files
files.download('Distribution_of_Passing_Yards.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Creating datasets by year for Wide Reciever data

In [12]:
wr_2017 = wr_data[wr_data['Year'] == 2017]
wr_2018 = wr_data[wr_data['Year'] == 2018]
wr_2019 = wr_data[wr_data['Year'] == 2019]
wr_2020 = wr_data[wr_data['Year'] == 2020]
wr_2021 = wr_data[wr_data['Year'] == 2021]
wr_2022 = wr_data[wr_data['Year'] == 2022]
allWrData = pd.concat([wr_2017, wr_2018, wr_2019, wr_2020, wr_2021, wr_2022])

Graphing the top 5 Wide Recievers by Yards in 2022

In [13]:
top5wr_2022 = wr_2022.sort_values(by='RecYds', ascending=False)[:5][['Player', 'RecYds']]


fig = go.Figure(data=[
    go.Bar(name='Recieving Yards', x=top5wr_2022['Player'], y=top5wr_2022['RecYds'], marker_color='green')
])


fig.update_layout(
    title='Top 5 Players by Recieving Yards in 2022',
    xaxis_title='Player',
    yaxis_title='Recieving Yards',
    xaxis_tickangle=0
)

fig.show()

fig.write_html("Top5wr2022.html")
from google.colab import files
files.download('Top5wr2022.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Graphing the top Wide Reciever by year in terms of recieving yards

In [14]:
topwr_2017 = wr_2017.sort_values(by='RecYds', ascending=False)[:1][['Player', 'RecYds', 'Year']]
topwr_2018 = wr_2018.sort_values(by='RecYds', ascending=False)[:1][['Player', 'RecYds', 'Year']]
topwr_2019 = wr_2019.sort_values(by='RecYds', ascending=False)[:1][['Player', 'RecYds', 'Year']]
topwr_2020 = wr_2020.sort_values(by='RecYds', ascending=False)[:1][['Player', 'RecYds', 'Year']]
topwr_2021 = wr_2021.sort_values(by='RecYds', ascending=False)[:1][['Player', 'RecYds', 'Year']]
topwr_2022 = wr_2022.sort_values(by='RecYds', ascending=False)[:1][['Player', 'RecYds', 'Year']]
topwr_data = pd.concat([topwr_2017, topwr_2018, topwr_2019, topwr_2020, topwr_2021, topwr_2022])

fig = go.Figure(data=[
    go.Bar(
        name='Receiving Yards',
        x=topwr_data['Year'],
        y=topwr_data['RecYds'],
        text=[f"{player}: {recyds} yards" for player, recyds in zip(topwr_data['Player'], topwr_data['RecYds'])],
        hoverinfo='text',
        marker_color='green'
    )
])

fig.update_layout(
    title='Top Receiving Player by Year',
    xaxis_title='Year',
    yaxis_title='Receiving Yards',
    xaxis_tickangle=0
)

fig.update_traces(textposition='none')

fig.show()

fig.write_html("Top_receiving_players.html")
from google.colab import files
files.download('Top_receiving_players.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Creating datasets by year for Running Back data

In [15]:
rb_2017 = rb_data[rb_data['Year'] == 2017]
rb_2018 = rb_data[rb_data['Year'] == 2018]
rb_2019 = rb_data[rb_data['Year'] == 2019]
rb_2020 = rb_data[rb_data['Year'] == 2020]
rb_2021 = rb_data[rb_data['Year'] == 2021]
rb_2022 = rb_data[rb_data['Year'] == 2022]

Graphing the top Running Back
 by year in terms of rushing yards

In [16]:
toprb_2017 = rb_2017.sort_values(by='RushYds', ascending=False)[:1][['Player', 'RushYds', 'Year']]
toprb_2018 = rb_2018.sort_values(by='RushYds', ascending=False)[:1][['Player', 'RushYds', 'Year']]
toprb_2019 = rb_2019.sort_values(by='RushYds', ascending=False)[:1][['Player', 'RushYds', 'Year']]
toprb_2020 = rb_2020.sort_values(by='RushYds', ascending=False)[:1][['Player', 'RushYds', 'Year']]
toprb_2021 = rb_2021.sort_values(by='RushYds', ascending=False)[:1][['Player', 'RushYds', 'Year']]
toprb_2022 = rb_2022.sort_values(by='RushYds', ascending=False)[:1][['Player', 'RushYds', 'Year']]
toprb_data = pd.concat([toprb_2017, toprb_2018, toprb_2019, toprb_2020, toprb_2021, toprb_2022])


fig = go.Figure(data=[
    go.Bar(
        name='Rushing Yards',
        x=toprb_data['Year'],
        y=toprb_data['RushYds'],
        text=[f"{player}: {rushyds} yards" for player, rushyds in zip(toprb_data['Player'], toprb_data['RushYds'])],
        hoverinfo='text',
        marker_color='red'
    )
])


fig.update_layout(
    title='Top Rushing Player by Year',
    xaxis_title='Year',
    yaxis_title='Rushing Yards',
    xaxis_tickangle=0
)


fig.update_traces(textposition='none')

fig.show()

fig.write_html("Top_rushing_players.html")
from google.colab import files
files.download('Top_rushing_players.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Creating datasets by year for Quarterback data and graphing the top Quarterback by passing yards for each year

In [17]:
qb_2017 = qb_data[qb_data['Year'] == 2017]
qb_2018 = qb_data[qb_data['Year'] == 2018]
qb_2019 = qb_data[qb_data['Year'] == 2019]
qb_2020 = qb_data[qb_data['Year'] == 2020]
qb_2021 = qb_data[qb_data['Year'] == 2021]
qb_2022 = qb_data[qb_data['Year'] == 2022]
allQbData = pd.concat([qb_2017, qb_2018, qb_2019, qb_2020, qb_2021, qb_2022])
qb_2017.head()

topqb_2017 = qb_2017.sort_values(by='PassingYds', ascending=False)[:1][['Player', 'PassingYds', 'Year']]
topqb_2018 = qb_2018.sort_values(by='PassingYds', ascending=False)[:1][['Player', 'PassingYds', 'Year']]
topqb_2019 = qb_2019.sort_values(by='PassingYds', ascending=False)[:1][['Player', 'PassingYds', 'Year']]
topqb_2020 = qb_2020.sort_values(by='PassingYds', ascending=False)[:1][['Player', 'PassingYds', 'Year']]
topqb_2021 = qb_2021.sort_values(by='PassingYds', ascending=False)[:1][['Player', 'PassingYds', 'Year']]
topqb_2022 = qb_2022.sort_values(by='PassingYds', ascending=False)[:1][['Player', 'PassingYds', 'Year']]
topqb_data = pd.concat([topqb_2017, topqb_2018, topqb_2019, topqb_2020, topqb_2021, topqb_2022])
topqb_data

fig = go.Figure(data=[
    go.Bar(
        name='Passing Yards',
        x=topqb_data['Year'],
        y=topqb_data['PassingYds'],
        text=[f"{player}: {passyds} yards" for player, passyds in zip(topqb_data['Player'], topqb_data['PassingYds'])],
        hoverinfo='text',
        marker_color='blue'
    )
])

fig.update_layout(
    title='Top Passing Player by Year',
    xaxis_title='Year',
    yaxis_title='Passing Yards',
    xaxis_tickangle=0
)


fig.update_traces(textposition='none')

fig.show()

fig.write_html("Top_passing_players.html")
from google.colab import files
files.download('Top_passing_players.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

To answer our second question regarding specific metrics to predict PPR we decided to only look at Quarterbacks, Recievers and Running backs with a 50% quartile level and over for the years 2017-2021. We begin by creating  datasets for Running Backs, Quarterbacks and Wide Recievers that exclude the year 2022 as well as only including data entries that meet our criteria above.


In [18]:
wrData2017_2021 = pd.concat([wr_2017, wr_2018, wr_2019, wr_2020, wr_2021])
wrData2017_2021['RecYds'].describe()

,RecYds
count,1079.000000
mean,376.660797
std,374.521978
min,-4.000000
25%,63.000000
50%,248.000000
75%,595.000000
max,1947.000000


As seen above the 50% quartile level for RecYds is 248, so we will modify the dataset above to only include Wide Recievers with at least 248 yards and over.

In [19]:
filtered_wrData2017_2021 = wrData2017_2021[wrData2017_2021['RecYds'] > 248]

Given this dataset we will use statistical code to compute the relationship between the wide reciever specific metric of Yards per Reception with PPR.

In [20]:
correlation, _ = pearsonr(filtered_wrData2017_2021['YR'], filtered_wrData2017_2021['PPR'] )

print(f"The correlation between Yards per Reception and PPR points is: {correlation:.2f}")


The correlation between Yards per Reception and PPR points is: 0.01


Scatter Plot displaying Yards per Reception against Fantasy Points

In [21]:
fig = go.Figure(data=go.Scatter(
    x=filtered_wrData2017_2021['YR'],  # X-axis data
    y=filtered_wrData2017_2021['PPR'],  # Y-axis data
    mode='markers',
    text=[f"{player}: {recyds} yards" for player, recyds in zip(filtered_wrData2017_2021['Player'], filtered_wrData2017_2021['YR'])],
    marker=dict(color='green', size=10)  # Customize markers
))

fig.update_layout(
    title='Scatter Plot of Yards per Reception vs Fantasy Points',
    xaxis_title='Yards per Reception',
    yaxis_title='Fantasy Points'
)

fig.show()

fig.write_html("Scatter_YR_PPR.html")
from google.colab import files
files.download('Scatter_YR_PPR.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

We will now use statistical code to compute the relationship between the wide reciever specific metric of Receiving Yards per Touchdown with PPR. To do this we must drop Nan values from the Receiving Yards per Touchdown column, so in effect we must drop wide receivers with 0 touchdowns.

In [22]:
cleaned_wrData2017_2021 = filtered_wrData2017_2021[filtered_wrData2017_2021['RecTD'] > 0]
cleaned_wrData2017_2021 = cleaned_wrData2017_2021.dropna(subset=['RecYds/TD'])


We will use statistical code to compute the relationship between the wide reciever specific metric of Receiving Yards per Touchdown with PPR.

In [23]:
correlation, _ = pearsonr(cleaned_wrData2017_2021['RecYds/TD'], cleaned_wrData2017_2021['PPR'] )

print(f"The correlation between Receiving Yards per Touchdown and PPR points is: {correlation:.2f}")

The correlation between Receiving Yards per Touchdown and PPR points is: -0.17


Scatter Plot displaying Receiving Yards per Touchdown against Fantasy Points

In [24]:
fig = go.Figure(data=go.Scatter(
    x=cleaned_wrData2017_2021['RecYds/TD'],  # X-axis data
    y=cleaned_wrData2017_2021['PPR'],  # Y-axis data
    mode='markers',   # Marker mode for scatter plot
    text=[f"{player}: {recyds} yards" for player, recyds in zip(cleaned_wrData2017_2021['Player'], cleaned_wrData2017_2021['RecYds/TD'])],
    marker=dict(color='green', size=10)  # Customize markers
))

fig.update_layout(
    title='Scatter Plot of Receiving Yards per Touchdown vs Fantasy Points',
    xaxis_title='Recieving Yards per Touchdown',
    yaxis_title='Fantasy Points'
)

fig.show()

fig.write_html("Scatter_YardsperTouchdown_PPR.html")
from google.colab import files
files.download('Scatter_YardsperTouchdown_PPR.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Analysis of Running Back data from 2017-2021, looking at the specifc metric of yards per attempt.

In [25]:
rbData2017_2021 = pd.concat([rb_2017, rb_2018, rb_2019, rb_2020, rb_2021])
rbData2017_2021['RushYds'].describe()

,RushYds
count,791.000000
mean,305.398230
std,355.665676
min,-3.000000
25%,24.000000
50%,157.000000
75%,493.500000
max,2027.000000


In [26]:
filtered_rbData2017_2021 = rbData2017_2021[rbData2017_2021['RushYds'] > 157]


We will use statistical code to compute the relationship between the running back specific metric of Rushing yards per attempt with PPR.

In [27]:
correlation, _ = pearsonr(filtered_rbData2017_2021['YA'], filtered_rbData2017_2021['PPR'] )

print(f"The correlation between Yards per Rush Attempt and PPR points is: {correlation:.2f}")

The correlation between Yards per Rush Attempt and PPR points is: 0.12


Scatter Plot displaying Rushing Yards per attempt against Fantasy Points

In [28]:
# Create the scatter plot
fig = go.Figure(data=go.Scatter(
    x=filtered_rbData2017_2021['YA'],  # X-axis data
    y=filtered_rbData2017_2021['PPR'],  # Y-axis data
    mode='markers',   # Marker mode for scatter plot
    text=[f"{player}: {recyds} yards" for player, recyds in zip(filtered_rbData2017_2021['Player'], filtered_rbData2017_2021['YA'])],
    marker=dict(color='red', size=10)  # Customize markers
))

# Update layout
fig.update_layout(
    title='Scatter Plot of Yards per Rush Attempt vs Fantasy Points',
    xaxis_title='Yards per Rush Attempt',
    yaxis_title='Fantasy Points'
)

# Show the plot
fig.show()

fig.write_html("Scatter_YA_PPR.html")
from google.colab import files
files.download('Scatter_YA_PPR.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Analysis of Running Back data from 2017-2021, looking at the specifc metric of Rushing Yards per Touchdown.

In [29]:
cleaned_rbData2017_2021 = filtered_rbData2017_2021[filtered_rbData2017_2021['RushTD'] > 0]
cleaned_rbData2017_2021 = cleaned_rbData2017_2021.dropna(subset=['RushYds/TD'])

We will use statistical code to compute the relationship between the running back specific metric of Rushing yards per touchdown with PPR.

In [30]:
correlation, _ = pearsonr(cleaned_rbData2017_2021['RushYds/TD'], cleaned_rbData2017_2021['PPR'] )

print(f"The correlation between Rushing Yards per Touchdown and PPR points is: {correlation:.2f}")

The correlation between Rushing Yards per Touchdown and PPR points is: -0.18


Scatter Plot displaying Rushing Yards per touchdown against Fantasy Points

In [31]:
# Create the scatter plot
fig = go.Figure(data=go.Scatter(
    x=cleaned_rbData2017_2021['RushYds/TD'],  # X-axis data
    y=filtered_rbData2017_2021['PPR'],  # Y-axis data
    mode='markers',   # Marker mode for scatter plot
    text=[f"{player}: {recyds} yards" for player, recyds in zip(cleaned_rbData2017_2021['Player'], cleaned_rbData2017_2021['RushYds/TD'])],
    marker=dict(color='red', size=10)  # Customize markers
))

# Update layout
fig.update_layout(
    title='Scatter Plot of Rushing Yards per Touchdown vs Fantasy Points',
    xaxis_title='Rushing Yards per Touchdown',
    yaxis_title='Fantasy Points'
)

# Show the plot
fig.show()

fig.write_html("Scatter_YardsRushingTouchdown_PPR.html")
from google.colab import files
files.download('Scatter_YardsRushingTouchdown_PPR.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Analysis of Quarterback data from 2017-2021, looking at the specifc metric of completion percentage.

In [32]:
qbData2017_2021 = pd.concat([qb_2017, qb_2018, qb_2019, qb_2020, qb_2021])
qbData2017_2021[['CompPct', 'Att']].describe()
filtered_qbData2017_2021 = qbData2017_2021[(qbData2017_2021['Att'] > 156) & (qbData2017_2021['CompPct'] > .63)]

We will use statistical code to compute the relationship between the quarterback specific metric of completion percentage with PPR.

In [33]:
correlation, _ = pearsonr(filtered_qbData2017_2021['CompPct'], filtered_qbData2017_2021['PPR'] )

print(f"The correlation between Completion Percentage and PPR points is: {correlation:.2f}")

The correlation between Completion Percentage and PPR points is: 0.19


Scatter Plot displaying Completion Percentage against Fantasy Points

In [34]:
# Create the scatter plot
fig = go.Figure(data=go.Scatter(
    x=filtered_qbData2017_2021['CompPct'],  # X-axis data
    y=filtered_qbData2017_2021['PPR'],  # Y-axis data
    mode='markers',   # Marker mode for scatter plot
    text=[f"{player}: {recyds} Percent" for player, recyds in zip(filtered_qbData2017_2021['Player'], filtered_qbData2017_2021['CompPct'])],
    marker=dict(color='blue', size=10)  # Customize markers
))

# Update layout
fig.update_layout(
    title='Scatter Plot of Completion Percentage vs Fantasy Points',
    xaxis_title='Completion Percentage',
    yaxis_title='Fantasy Points'
)

# Show the plot
fig.show()

fig.write_html("Scatter_CompPCT_PPR.html")
from google.colab import files
files.download('Scatter_CompPCT_PPR.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>